In [ ]:
'''
Introdução à Econometria - Uma abordagem moderna (Tradução da 6 edição norte-americana)
Autor: WOOLDRIDGE, J. M.
Editora: CENGAGE LEARNING

Cap. 6: Análise de regressão múltipla: problemas adicionais (Multiple Regression Analysis: Further Issues)
Exemplo 6.5: Intervalo de confiança de colgpa previsto 
             (CONFIDENCE INTERVAL FOR PREDICTED COLLEGE GPA)
             
Arquivo com os dados: gpa2.xls

Arquivo com dados em:
http://students.cengage.com.br/dashboard/private/livroView.jsf;jsessionid=95E9AD889A4A4B7ABBD2A5251F1E14BE?id=104577

Em caso de dúvidas ou problemas, solicitamos, por gentileza, entrar em contato pelo e-mail:
python.economia@gmail.com
'''

In [1]:
import pandas as pd
import statsmodels.formula.api as smf

In [2]:
df = pd.read_excel('gpa2.xls',
                   header=None,
                   usecols=[0, 2, 5, 7, 11],
                   names=['sat', 'colgpa', 'hsize', 'hsperc', 'hsizesq'])

In [3]:
df.head()

,sat,colgpa,hsize,hsperc,hsizesq
0,920,2.04,0.10,40.00000,0.0100
1,1170,4.00,9.40,20.31915,88.3600
2,810,1.78,1.19,35.29412,1.4161
3,940,2.42,5.71,44.13310,32.6041
4,1180,2.61,2.14,40.18692,4.5796


### Comando sugeridos

#### Regressão por Mínimos Quadrados Ordinários (MQO ou OLS na sigla em inglês)

In [4]:
modelo = smf.ols(formula='colgpa ~ sat + hsperc + hsize + hsizesq', data=df)
reg = modelo.fit()

In [5]:
# Tabela de resultados
tabela = pd.DataFrame()
tabela['coef'] = round(reg.params, 5)
tabela['std err'] = round(reg.bse, 5)

n_obs = reg.nobs
sigma_chapeu = reg.mse_resid ** 0.5
r_quadrado = reg.rsquared
r_quadrado_ajustado = reg.rsquared_adj

print(f'Variável dependente: colgpa\n')
print(tabela)
print(f'\nn = {int(n_obs)}, R2 = {round(r_quadrado, 3)}, R2 ajustado: {round(r_quadrado_ajustado, 3)}, \
sigma_chapeu = {round(sigma_chapeu, 3)}' )

Variável dependente: colgpa

              coef  std err
Intercept  1.49265  0.07534
sat        0.00149  0.00007
hsperc    -0.01386  0.00056
hsize     -0.06088  0.01650
hsizesq    0.00546  0.00227

n = 4137, R2 = 0.278, R2 ajustado: 0.277, sigma_chapeu = 0.56


In [6]:
# Intervalo de confiança de 95% para o colgpa esperado
exog = {'sat': 1200, 'hsperc': 30, 'hsize': 5, 'hsizesq': 25}

intervalo_confianca_colgpa = reg.get_prediction(exog)
intervalo_confianca_colgpa.conf_int(obs=False, alpha=0.05)    # Para obter o IC de 95% no exemplo 6.6, basta colocar "obs=True"

array([[2.66110422, 2.73904676]])

#### Intervalo de Confiança de 95% da média do GPA calculado pelo procedimento sugerido no livro

In [7]:
sat = 1200
hsperc = 30
hsize = 5

df['sat0'] = df['sat'] - sat
df['hsperc0'] = df['hsperc'] - hsperc
df['hsize0'] = df['hsize'] - hsize
df['hsizesq0'] = df['hsizesq'] - hsize ** 2

modelo_predict = smf.ols('colgpa ~ sat0 + hsperc0 + hsize0 + hsizesq0', data=df)
reg_predict = modelo_predict.fit()

In [8]:
# Tabela de resultados
tabela_predict = pd.DataFrame()
tabela_predict['coef'] = round(reg_predict.params, 5)
tabela_predict['std err'] = round(reg_predict.bse, 5)

n_obs_predict = reg_predict.nobs
sigma_chapeu_predict = reg_predict.mse_resid ** 0.5
r_quadrado_predict = reg_predict.rsquared
r_quadrado_ajustado_predict = reg_predict.rsquared_adj

print(f'Variável dependente: colgpa\n')
print(tabela_predict)
print(f'\nn = {int(n_obs_predict)}, R2 = {round(r_quadrado_predict, 3)}, R2 ajustado: {round(r_quadrado_ajustado_predict, 3)}, \
sigma_chapeu = {round(sigma_chapeu_predict, 3)}' )

Variável dependente: colgpa

              coef  std err
Intercept  2.70008  0.01988
sat0       0.00149  0.00007
hsperc0   -0.01386  0.00056
hsize0    -0.06088  0.01650
hsizesq0   0.00546  0.00227

n = 4137, R2 = 0.278, R2 ajustado: 0.277, sigma_chapeu = 0.56


In [9]:
# Intervalo de confiança de 95% para o colgpa esperado
inferior = reg_predict.params['Intercept'] - 1.96 * reg_predict.bse['Intercept'] 
superior = reg_predict.params['Intercept'] + 1.96 * reg_predict.bse['Intercept']

intervalo_confianca_95 = (inferior, superior)

intervalo_confianca_95

(2.6611149182623732, 2.7390360554797146)

In [ ]:
'''
Observação:
O erro padrão de hsize0 consta no livro como 0,1650, quando na realidade é 0,01650 (ver na tabela de resultados acima)
'''